In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import yfinance as yf

In [ ]:
def get_random_weights(l:int)->np.ndarray:
  return np.random.dirichlet(np.ones(l))

def get_weights(data):
  vol = data.pct_change().std()
  weights = (vol)/np.sum(vol)
  return weights

In [ ]:
n_steps = 252
n_paths = 30
T = 1
market_ticker = ["^GSPC"]
portfolio_value = 1000
tickers = ["GOOGL", "NVDA",
           "FB", "PLTR", 
           "WFC", "DIS", 
           "LUV", "PFE",
           "COKE", "CAT"]


In [ ]:
data = yf.download(tickers + market_ticker, start="2025-01-01", end="2026-01-01")["Close"]

In [ ]:
def get_params(data):
  S0 = data.values[-1]

  log_returns = np.log(data / data.shift(1)).dropna()

  mu = log_returns.mean().values
  sigma = log_returns.std().values
  corr = data.corr()

  return S0, mu, sigma, corr

In [ ]:
ticker_data = data.drop(market_ticker, axis=1)
market = data.drop(tickers, axis=1)

S0, mu, sigma, corr = get_params(ticker_data)
weights = np.asarray(get_weights(ticker_data))
print(np.sum(weights))
print(weights)

In [ ]:
def scale_to_V0(S0_raw: np.ndarray, V0: float, weights: np.ndarray):
  S0_raw = np.asarray(S0_raw)
  weights = np.asarray(weights)

  pf_v =  weights @ S0_raw
  scale = V0/pf_v
  return S0_raw * scale
  

def gbm_paths(
    S0_raw: np.ndarray, mu: list, sigma: list, weights: np.ndarray,
    corr: np.ndarray, T: float, N: int, n_paths: int, V0: float
  ):
  S0 = scale_to_V0(S0_raw, V0, weights)
  mu = np.asarray(mu)
  sigma = np.asarray(sigma)

  n_assets = len(S0)
  dt = T/N

  eps = np.finfo(float).eps
  eigvals, eigvec = np.linalg.eigh(corr)
  eigvals = np.clip(eigvals, eps, None)
  corr_pd = eigvec @ np.diag(eigvals) @ eigvec.T

  L = np.linalg.cholesky(corr_pd)
  Z = np.random.standard_normal((n_paths, N, n_assets))
  Z_corr = Z @ L.T

  W = np.cumsum(np.sqrt(dt) * Z_corr, axis=1)

  t = np.linspace(dt, T, N).reshape(1, N, 1)

  paths = S0 * np.exp(
      (mu - 0.5 * sigma**2) * t + sigma * W
  )
  return paths


def get_returns(data: pd.DataFrame):
  return data.pct_change().dropna()

In [ ]:
paths = gbm_paths(S0, mu, sigma, corr=corr, T=1, N=n_steps, n_paths=n_paths, weights=weights, V0=portfolio_value)
portfolios = pd.DataFrame((paths*weights).sum(axis=2).T)

In [ ]:
plt.plot(portfolios)
plt.title("Sample portfolio paths")
plt.show()

In [ ]:
market_returns = get_returns(market)
portfolio_returns = get_returns(portfolios)

In [ ]:
class AlphaEstimator:
  def __init__(self, market_returns, portfolio_returns):
    self.market_returns = market_returns
    self.portfolio_returns = portfolio_returns
  
  def process_portfolio_paths(self):
    pass

  def estimate_alpha(self):
    pass

  def plot_range_distribution(self):
    pass

In [ ]:
estimator = AlphaEstimator(
    market_returns=market_returns,
    portfolio_returns=portfolio_returns
)